<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/BigQuery_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [28]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [0]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [31]:
# Display query output immediately

%%bigquery --project bigquery-207917
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [0]:
# Save output in a variable `df`

%%bigquery --project bigquery-207917 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [0]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [0]:
project_id = '[bigquery-207917]'

### Sample approximately 2000 random rows

In [37]:
from google.cloud import bigquery

client = bigquery.Client(project='bigquery-207917')

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [38]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,1997.000000,1997.000000,1997.000000,1997.000000,1997.000000,1997.000000,1997.000000,1914.000000,1914.000000,1558.000000,1558.000000,728.000000,728.000000,1811.000000,1811.000000,1969.000000,1969.000000,1938.000000,265.000000,1994.000000,1820.000000,117.000000
mean,499950.171758,89980.867301,1987.408613,6.579369,15.712569,52.310966,13.114171,41.919906,13.101881,1014.731323,11.408858,969.014012,12.354396,12.397184,12.723357,6.863128,13.104622,12.256192,25.394717,43.831444,0.073368,9.839316
std,300598.987968,26859.626061,15.935846,3.448350,8.876096,23.604543,7.952639,22.101849,7.949301,9.601249,7.530906,65.627123,7.911700,9.984648,7.856180,4.922885,7.920542,6.912485,8.906346,23.495573,0.335501,9.707462
min,10280.000000,6.000000,1931.000000,1.000000,1.000000,-57.000000,4.000000,-61.099998,4.000000,905.299988,4.000000,607.000000,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,3.900000,-69.000000,0.000000,0.400000
25%,234160.000000,99999.000000,1978.000000,4.000000,8.000000,37.500000,7.000000,29.125000,7.000000,1009.700012,6.000000,952.249985,7.000000,6.400000,6.500000,3.500000,7.000000,7.800000,19.799999,30.900000,0.000000,3.500000
50%,517650.000000,99999.000000,1990.000000,7.000000,16.000000,55.400002,8.000000,43.850000,8.000000,1014.299988,8.000000,995.350006,8.000000,9.600000,8.000000,5.800000,8.000000,11.100000,23.900000,46.400002,0.000000,7.100000
75%,725065.000000,99999.000000,2000.000000,10.000000,24.000000,71.000000,24.000000,57.375001,24.000000,1019.900024,20.000000,1009.500000,23.000000,14.900000,23.000000,9.100000,23.000000,15.500000,29.900000,61.000000,0.000000,14.200000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,97.300003,24.000000,84.099998,24.000000,1053.000000,24.000000,1035.300049,24.000000,99.400002,24.000000,45.200001,24.000000,56.900002,62.900002,96.800003,8.150000,56.700001


### View the first 10 rows

In [39]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,368700,99999,1935,8,31,75.199997,4,NaN,NaN,NaN,NaN,NaN,NaN,12.4,4.0,5.0,4.0,8.9,NaN,57.000000,False,None,None,0.00,NaN,False,False,False,False,False,False
1,401800,99999,1982,6,6,68.599998,17,57.799999,17.0,NaN,NaN,NaN,NaN,6.2,17.0,4.0,17.0,11.8,NaN,59.900002,True,None,None,0.00,NaN,False,False,False,False,False,False
2,579060,99999,1962,8,4,79.199997,4,74.500000,4.0,1003.500000,4.0,NaN,NaN,21.0,4.0,0.0,4.0,NaN,NaN,75.000000,True,None,None,0.06,NaN,False,False,False,False,False,False
3,479450,99999,1988,7,13,83.800003,12,77.400002,12.0,1012.299988,7.0,NaN,NaN,11.0,12.0,2.7,12.0,5.1,NaN,75.199997,True,None,None,0.12,NaN,True,True,True,True,True,True
4,561960,99999,1993,8,8,80.000000,7,71.099998,7.0,1001.799988,7.0,NaN,NaN,9.1,7.0,4.2,7.0,5.8,NaN,72.000000,True,None,None,1.22,NaN,False,False,False,False,False,False
5,969250,99999,1994,4,4,81.699997,7,75.400002,7.0,1011.400024,7.0,NaN,NaN,9.3,7.0,1.7,7.0,5.1,NaN,74.099998,False,None,None,0.00,NaN,False,False,False,False,False,False
6,517300,99999,1974,9,20,67.699997,7,49.500000,7.0,1013.400024,7.0,901.299988,4.0,12.4,7.0,3.3,7.0,7.8,NaN,51.799999,False,None,None,0.00,NaN,False,False,False,False,False,False
7,141210,99999,2005,1,3,33.400002,7,32.299999,7.0,1026.900024,7.0,999.299988,7.0,0.1,4.0,2.8,7.0,3.9,NaN,30.700001,True,None,None,0.15,NaN,True,True,True,True,True,True
8,637930,99999,1988,12,5,76.400002,10,66.599998,10.0,1012.400024,8.0,NaN,NaN,22.5,9.0,4.8,9.0,6.0,NaN,69.099998,False,None,None,0.00,NaN,False,False,False,False,False,False
9,358490,99999,2003,3,4,18.200001,9,4.200000,9.0,1035.599976,9.0,NaN,NaN,12.4,9.0,2.6,9.0,3.9,NaN,4.100000,False,None,None,0.00,NaN,False,False,False,False,False,False


In [40]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
1877,478300,1989,8,21,8.15
1850,442180,1961,8,8,4.06
184,987550,1975,12,13,3.54
941,599970,2001,12,13,3.07
685,476120,2002,11,9,2.72
414,942030,1991,2,23,2.52
1066,483250,2005,8,31,2.19
576,988360,1993,7,9,2.17
1837,804340,1993,11,1,2.09
1921,726116,2008,10,26,2.06


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [43]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id='bigquery-207917', dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028
